In [2]:
import open3d as o3d
import numpy as np
import pymeshlab

In [100]:
mesh = o3d.io.read_triangle_mesh(r"G:\VS2022Projects\arap-volume-tracking-main\data\scene\meshes\good/mesh_0001.obj")
mesh.compute_vertex_normals()
#o3d.visualization.draw_geometries([mesh])
aabb = mesh.get_axis_aligned_bounding_box()
min_bound = aabb.min_bound
max_bound = aabb.max_bound

# Define the number of blocks along each dimension
num_blocks = [8, 8, 6]  # Number of divisions in x, y, z

block_size = [(max_bound[i] - min_bound[i]) / num_blocks[i] for i in range(3)]

# Generate block boundaries
lines = []
points = []

for i in range(num_blocks[0] + 1):  # X divisions
    for j in range(num_blocks[1] + 1):  # Y divisions
        for k in range(num_blocks[2] + 1):  # Z divisions
            # Current grid point
            x = min_bound[0] + i * block_size[0]
            y = min_bound[1] + j * block_size[1]
            z = min_bound[2] + k * block_size[2]
            points.append([x, y, z])

# Map points to indices for lines
points = np.array(points)
idx = lambda i, j, k: i * (num_blocks[1] + 1) * (num_blocks[2] + 1) + j * (num_blocks[2] + 1) + k

# Add lines for block edges
for i in range(num_blocks[0] + 1):
    for j in range(num_blocks[1] + 1):
        for k in range(num_blocks[2]):
            lines.append([idx(i, j, k), idx(i, j, k + 1)])
for i in range(num_blocks[0] + 1):
    for j in range(num_blocks[1]):
        for k in range(num_blocks[2] + 1):
            lines.append([idx(i, j, k), idx(i, j + 1, k)])
for i in range(num_blocks[0]):
    for j in range(num_blocks[1] + 1):
        for k in range(num_blocks[2] + 1):
            lines.append([idx(i, j, k), idx(i + 1, j, k)])

# Create LineSet for visualization
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(points)
line_set.lines = o3d.utility.Vector2iVector(lines)

# Visualize mesh with block boundaries
o3d.visualization.draw_geometries([mesh, line_set])

In [3]:
def get_selected_parts(mesh, dynamic_indices, static_indices):
    selected_vertex_indices = dynamic_indices 
    
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    selected_indices_set = set(selected_vertex_indices)
    
    new_vertex_indices = {}
    new_index_counter = 0
    
    new_vertices = []
    new_triangles = []
    
    for tri in triangles:
        in_selected = [v in selected_indices_set for v in tri]
        if sum(in_selected) >= 1: 
            new_triangle = []
            for v in tri:
                if v not in new_vertex_indices:
                    new_vertex_indices[v] = new_index_counter
                    new_vertices.append(vertices[v])
                    new_index_counter += 1
                new_triangle.append(new_vertex_indices[v])
            new_triangles.append(new_triangle)
    
    dynamic_mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(new_vertices),
        triangles=o3d.utility.Vector3iVector(new_triangles)
    )
    
    dynamic_mesh.compute_vertex_normals()
    
    dynamic_mesh.paint_uniform_color([0.4, 0.4, 0.4])
    
    static_mesh = mesh.select_by_index(static_indices)
    static_mesh.compute_vertex_normals()
    return dynamic_mesh, static_mesh


In [88]:
import open3d as o3d
import numpy as np
mesh1 = r"G:\ChromeDownloads\Raw\Mesh/preprocessed/answering_questions_80.obj"
mesh7 = r"G:\ChromeDownloads\Raw\Mesh/preprocessed/answering_questions_89.obj"
# Parameters
num_blocks = [5, 5, 5]  
dynamic_threshold = 0.9 
comparison_tolerance = 0.000005
mesh_files = []
for i in range (0, 10):
    mesh_i = fr"G:\ChromeDownloads\Raw\Mesh/preprocessed/answering_questions_8{i:01}.obj"
    mesh_files.append(mesh_i)
mesh_files = [mesh1, mesh7]  
meshes = [o3d.io.read_triangle_mesh(file) for file in mesh_files]

aabb = meshes[0].get_axis_aligned_bounding_box()
for mesh in meshes[1:]:
    aabb += mesh.get_axis_aligned_bounding_box()

min_bound = aabb.min_bound
max_bound = aabb.max_bound
block_size = [(max_bound[i] - min_bound[i]) / num_blocks[i] for i in range(3)]

def get_block_index(point):
    return tuple(int((point[i] - min_bound[i]) / block_size[i]) for i in range(3))


def is_block_dynamic(points1, points2, threshold, tolerance):
    if not len(points1) or not len(points2):  
        return True
    
    points1 = np.array(points1)
    points2 = np.array(points2)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points2)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    differing_points = 0

    for point in points1:
        [k, idx, dist] = pcd_tree.search_knn_vector_3d(point, 1)
        if k == 0 or dist[0] > tolerance: 
            differing_points += 1

    #print(differing_points)
    return (differing_points / len(points1)) > threshold

occupancy_maps = [{} for _ in meshes]

for t, mesh in enumerate(meshes):
    points = np.asarray(mesh.vertices)
    for point in points:
        block_index = get_block_index(point)
        if block_index not in occupancy_maps[t]:
            occupancy_maps[t][block_index] = []
        occupancy_maps[t][block_index].append(point)
 
all_blocks = set.union(*(set(occupancy.keys()) for occupancy in occupancy_maps))
dynamic_blocks = set()
static_blocks = set()

for block in all_blocks:
    is_dynamic = False
    for t in range(1, len(meshes)):
        if is_block_dynamic(
            occupancy_maps[t - 1].get(block, []),
            occupancy_maps[t].get(block, []),
            threshold=dynamic_threshold,
            tolerance=comparison_tolerance,
        ):
            is_dynamic = True
            break
    if is_dynamic:
        dynamic_blocks.add(block)
    else:
        static_blocks.add(block)

print(dynamic_blocks)
for t, mesh in enumerate(meshes):
    static_points = []
    dynamic_points = []
    points = np.asarray(mesh.vertices)
    for i, point in enumerate(points):
        block_index = get_block_index(point)
        if block_index in static_blocks:
            static_points.append(i)
        elif block_index in dynamic_blocks:
            dynamic_points.append(i)
            
    dynamic_mesh, static_mesh = get_selected_parts(mesh, dynamic_points, static_points)
    
    o3d.visualization.draw_geometries([dynamic_mesh])
    o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}_raw.obj', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}.obj')
    #ms.meshing_remove_connected_component_by_face_number(mincomponentsize = 1000)
    ms.save_current_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}.obj')

    mesh_raw = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic\mesh_0{t:03}_raw.obj')
    mesh = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic\mesh_0{t:03}.obj')
    
    vertices_raw = np.asarray(mesh_raw.vertices)
    vertices = np.asarray(mesh.vertices)
    
    diff_vertices_mask = np.any(vertices_raw[:, None] != vertices, axis=2).all(axis=1)
    
    diff_vertices = vertices_raw[diff_vertices_mask]
    diff_indices = np.where(diff_vertices_mask)[0]  
    
    diff_mesh = mesh_raw.select_by_index(diff_indices)
    
    static_mesh  = static_mesh + diff_mesh
    o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\static/mesh_0{t:03}_raw.obj', static_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    

#o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0000', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)


{(2, 2, 2), (0, 1, 3), (2, 1, 3), (0, 3, 3), (1, 2, 2), (2, 3, 0), (0, 2, 2), (2, 3, 3), (1, 1, 3), (0, 1, 2), (5, 0, 3), (0, 3, 5), (0, 3, 2), (4, 4, 3), (0, 2, 3), (1, 1, 2), (0, 5, 2), (0, 1, 1), (2, 2, 3), (0, 3, 1), (1, 2, 0), (1, 2, 3), (3, 3, 0), (4, 3, 0), (1, 1, 4)}
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.


## Remove planar surfaces with polygon numbers (not good)

In [33]:
import open3d as o3d
import numpy as np
mesh1 = r"G:\ChromeDownloads\Raw\Mesh/preprocessed/answering_questions_80.obj"
mesh7 = r"G:\ChromeDownloads\Raw\Mesh/preprocessed/answering_questions_89.obj"
# Parameters
num_blocks = [5, 5, 5]  
dynamic_threshold = 0.9 
comparison_tolerance = 0.000005
mesh_files = []
for i in range (0, 10):
    mesh_i = fr"G:\ChromeDownloads\Raw\Mesh/preprocessed/answering_questions_8{i:01}.obj"
    mesh_files.append(mesh_i)
#mesh_files = [mesh1, mesh7]  
meshes = [o3d.io.read_triangle_mesh(file) for file in mesh_files]

aabb = meshes[0].get_axis_aligned_bounding_box()
for mesh in meshes[1:]:
    aabb += mesh.get_axis_aligned_bounding_box()

min_bound = aabb.min_bound
max_bound = aabb.max_bound
block_size = [(max_bound[i] - min_bound[i]) / num_blocks[i] for i in range(3)]

def get_block_index(point):
    return tuple(int((point[i] - min_bound[i]) / block_size[i]) for i in range(3))


def is_block_dynamic(points1, points2, threshold, tolerance):
    if not len(points1) or not len(points2):  
        return True
    
    points1 = np.array(points1)
    points2 = np.array(points2)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points2)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    differing_points = 0

    for point in points1:
        [k, idx, dist] = pcd_tree.search_knn_vector_3d(point, 1)
        if k == 0 or dist[0] > tolerance: 
            differing_points += 1

    #print(differing_points)
    return (differing_points / len(points1)) > threshold

occupancy_maps = [{} for _ in meshes]

for t, mesh in enumerate(meshes):
    points = np.asarray(mesh.vertices)
    for point in points:
        block_index = get_block_index(point)
        if block_index not in occupancy_maps[t]:
            occupancy_maps[t][block_index] = []
        occupancy_maps[t][block_index].append(point)
 
all_blocks = set.union(*(set(occupancy.keys()) for occupancy in occupancy_maps))
dynamic_blocks = set()
static_blocks = set()

for block in all_blocks:
    is_dynamic = False
    for t in range(1, len(meshes)): # there is a bug...
        if is_block_dynamic(
            occupancy_maps[0].get(block, []),
            occupancy_maps[len(meshes)-1].get(block, []),
            threshold=dynamic_threshold,
            tolerance=comparison_tolerance,
        ):
            is_dynamic = True
            break
    if is_dynamic:
        dynamic_blocks.add(block)
    else:
        static_blocks.add(block)

print(dynamic_blocks)
for t, mesh in enumerate(meshes):
    static_points = []
    dynamic_points = []
    points = np.asarray(mesh.vertices)
    for i, point in enumerate(points):
        block_index = get_block_index(point)
        if block_index in static_blocks:
            static_points.append(i)
        elif block_index in dynamic_blocks:
            dynamic_points.append(i)
            
    dynamic_mesh, static_mesh = get_selected_parts(mesh, dynamic_points, static_points)
    
    o3d.visualization.draw_geometries([dynamic_mesh])
    o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}_raw.obj', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}_raw.obj')
    ms.meshing_remove_connected_component_by_face_number(mincomponentsize = 1000)
    ms.save_current_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}.obj')

    mesh_raw = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic\mesh_0{t:03}_raw.obj')
    mesh = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic\mesh_0{t:03}.obj')
    
    vertices_raw = np.asarray(mesh_raw.vertices)
    vertices = np.asarray(mesh.vertices)
    
    diff_vertices_mask = np.any(vertices_raw[:, None] != vertices, axis=2).all(axis=1)
    
    diff_vertices = vertices_raw[diff_vertices_mask]
    diff_indices = np.where(diff_vertices_mask)[0]  
    
    diff_mesh = mesh_raw.select_by_index(diff_indices)
    
    static_mesh  = static_mesh + diff_mesh
    o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\static/mesh_0{t:03}_raw.obj', static_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    

#o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0000', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)


{(2, 2, 2), (0, 1, 3), (2, 1, 3), (0, 3, 3), (1, 2, 2), (2, 3, 0), (0, 2, 2), (2, 3, 3), (1, 1, 3), (0, 1, 2), (5, 0, 3), (0, 3, 2), (4, 4, 3), (0, 2, 3), (1, 1, 2), (0, 5, 2), (0, 1, 1), (2, 2, 3), (0, 3, 1), (1, 2, 0), (4, 4, 5), (1, 2, 3), (3, 3, 0), (4, 3, 0), (1, 1, 4)}
[Open3D WARNING] Write OBJ can not include triangle normals.
[Open3D WARNING] Write OBJ can not include triangle normals.


KeyboardInterrupt: 

In [99]:
mesh0 = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0000_raw.obj')
mesh1 = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0000.obj')

mesh0.compute_vertex_normals()
mesh1.compute_vertex_normals()
mesh0.paint_uniform_color([0.4, 0.4, 0.4])
o3d.visualization.draw_geometries([mesh0])

In [4]:
import open3d as o3d
import numpy as np
from collections import defaultdict

def build_triangle_adjacency(mesh):
    triangles = np.asarray(mesh.triangles)
    edges_to_triangles = defaultdict(list)

    for tri_idx, triangle in enumerate(triangles):
        edges = [
            tuple(sorted([triangle[0], triangle[1]])),
            tuple(sorted([triangle[1], triangle[2]])),
            tuple(sorted([triangle[2], triangle[0]])),
        ]
        for edge in edges:
            edges_to_triangles[edge].append(tri_idx)

    triangle_neighbors = defaultdict(set)
    for edge, tri_indices in edges_to_triangles.items():
        if len(tri_indices) > 1:  
            for tri_a in tri_indices:
                for tri_b in tri_indices:
                    if tri_a != tri_b:
                        triangle_neighbors[tri_a].add(tri_b)

    return triangle_neighbors

def find_connected_components(mesh):
    triangle_adjacency = build_triangle_adjacency(mesh)
    triangles = np.asarray(mesh.triangles)

    visited = set()
    connected_components = []

    for i in range(len(triangles)):
        if i in visited:
            continue

        component = [i]
        visited.add(i)

        queue = [i]
        while queue:
            current = queue.pop(0)
            for neighbor in triangle_adjacency[current]:
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
                    component.append(neighbor)

        connected_components.append(component)

    return connected_components

def is_planar_group(normals, component, tolerance_deg):
    reference_normal = normals[component[0]]
    for triangle_idx in component:
        normal = normals[triangle_idx]
        angle = np.arccos(np.clip(np.dot(reference_normal, normal), -1.0, 1.0))
        angle_deg = np.degrees(angle)
        if angle_deg > tolerance_deg:
            return False
    return True

def extract_mesh_from_group(mesh, group):
    triangles = np.asarray(mesh.triangles)
    vertices = np.asarray(mesh.vertices)

    sub_triangles = triangles[group]
    unique_vertex_indices = np.unique(sub_triangles)
    vertex_map = {old_idx: new_idx for new_idx, old_idx in enumerate(unique_vertex_indices)}

    remapped_triangles = np.array([[vertex_map[vid] for vid in triangle] for triangle in sub_triangles])
    sub_vertices = vertices[unique_vertex_indices]

    sub_mesh = o3d.geometry.TriangleMesh()
    sub_mesh.vertices = o3d.utility.Vector3dVector(sub_vertices)
    sub_mesh.triangles = o3d.utility.Vector3iVector(remapped_triangles)

    return sub_mesh

def find_planar_surfaces(mesh, tolerance_deg=5):
    mesh.compute_triangle_normals()
    normals = np.asarray(mesh.triangle_normals)

    connected_components = find_connected_components(mesh)
    print(connected_components.__len__())

    planar_groups = []
    for component in connected_components:
        if is_planar_group(normals, component, tolerance_deg):
            planar_groups.append(component)

    return planar_groups


mesh = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0000_raw.obj')


planar_groups = find_planar_surfaces(mesh, tolerance_deg=100)
print(planar_groups.__len__())

planar_meshes = [extract_mesh_from_group(mesh, group) for group in planar_groups]

deleted_mesh = o3d.geometry.TriangleMesh()
for i, planar_mesh in enumerate(planar_meshes):
    deleted_mesh += planar_mesh
    
o3d.visualization.draw_geometries([deleted_mesh])

31
6


## answering

In [7]:
import open3d as o3d
import numpy as np
mesh1 = r"G:\ChromeDownloads\Raw\Mesh/preprocessed/answering_questions_80.obj"
mesh7 = r"G:\ChromeDownloads\Raw\Mesh/preprocessed/answering_questions_89.obj"
# Parameters
num_blocks = [5, 5, 5]  
dynamic_threshold = 0.9
comparison_tolerance = 0.000005
mesh_files = []
for i in range (0, 10):
    mesh_i = fr"G:\ChromeDownloads\Raw\Mesh/preprocessed/answering_questions_8{i:01}.obj"
    mesh_files.append(mesh_i)
#mesh_files = [mesh1, mesh7]  
meshes = [o3d.io.read_triangle_mesh(file) for file in mesh_files]

aabb = meshes[0].get_axis_aligned_bounding_box()
for mesh in meshes[1:]:
    aabb += mesh.get_axis_aligned_bounding_box()

min_bound = aabb.min_bound
max_bound = aabb.max_bound
block_size = [(max_bound[i] - min_bound[i]) / num_blocks[i] for i in range(3)]

def get_block_index(point):
    return tuple(int((point[i] - min_bound[i]) / block_size[i]) for i in range(3))


def is_block_dynamic(points1, points2, threshold, tolerance):
    if not len(points1) or not len(points2):  
        return True
    
    points1 = np.array(points1)
    points2 = np.array(points2)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points2)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    differing_points = 0

    for point in points1:
        [k, idx, dist] = pcd_tree.search_knn_vector_3d(point, 1)
        if k == 0 or dist[0] > tolerance: 
            differing_points += 1

    #print(differing_points)
    return (differing_points / len(points1)) > threshold

occupancy_maps = [{} for _ in meshes]

for t, mesh in enumerate(meshes):
    points = np.asarray(mesh.vertices)
    for point in points:
        block_index = get_block_index(point)
        if block_index not in occupancy_maps[t]:
            occupancy_maps[t][block_index] = []
        occupancy_maps[t][block_index].append(point)
 
all_blocks = set.union(*(set(occupancy.keys()) for occupancy in occupancy_maps))
dynamic_blocks = set()
static_blocks = set()

for block in all_blocks:
    is_dynamic = False
    for t in range(1, len(meshes)): # there is a bug...
        if is_block_dynamic(
            occupancy_maps[0].get(block, []),
            occupancy_maps[len(meshes)-1].get(block, []),
            threshold=dynamic_threshold,
            tolerance=comparison_tolerance,
        ):
            is_dynamic = True
            break
    if is_dynamic:
        dynamic_blocks.add(block)
    else:
        static_blocks.add(block)

print(dynamic_blocks)
for t, mesh in enumerate(meshes):
    static_points = []
    dynamic_points = []
    points = np.asarray(mesh.vertices)
    for i, point in enumerate(points):
        block_index = get_block_index(point)
        if block_index in static_blocks:
            static_points.append(i)
        elif block_index in dynamic_blocks:
            dynamic_points.append(i)
            
    dynamic_mesh, static_mesh = get_selected_parts(mesh, dynamic_points, static_points)
    
    o3d.visualization.draw_geometries([dynamic_mesh])
    o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}_raw.obj', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

    
    planar_groups = find_planar_surfaces(dynamic_mesh, tolerance_deg=100)
    print(planar_groups.__len__())
    
    planar_meshes = [extract_mesh_from_group(dynamic_mesh, group) for group in planar_groups]
    
    deleted_mesh = o3d.geometry.TriangleMesh()
    for i, planar_mesh in enumerate(planar_meshes):
        deleted_mesh += planar_mesh
        
    #o3d.visualization.draw_geometries([deleted_mesh])

    o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}_deleted_normal.obj', deleted_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    mesh_raw = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic\mesh_0{t:03}_raw.obj')
    mesh = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic\mesh_0{t:03}_deleted_normal.obj')
    vertices_raw = np.asarray(mesh_raw.vertices)
    vertices = np.asarray(mesh.vertices)
    
    diff_vertices_mask = np.any(vertices_raw[:, None] != vertices, axis=2).all(axis=1)
    
    diff_vertices = vertices_raw[diff_vertices_mask]
    diff_indices = np.where(diff_vertices_mask)[0]  
    
    diff_mesh = mesh_raw.select_by_index(diff_indices)
    #print(diff_mesh)
    o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}_before_delete_pieces.obj', diff_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}_before_delete_pieces.obj')
    ms.meshing_remove_connected_component_by_face_number(mincomponentsize = 1200)
    ms.save_current_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}.obj')

    mesh_remove_face_number = o3d.io.read_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic\mesh_0{t:03}.obj')
    #o3d.visualization.draw_geometries([mesh_remove_face_number])
    
    vertices_with_pieces = np.asarray(diff_mesh.vertices)
    vertices_without_pieces = np.asarray(mesh_remove_face_number.vertices)
    
    diff_vertices_mask = np.any(vertices_with_pieces[:, None] != vertices_without_pieces, axis=2).all(axis=1)
    
    diff_vertices = vertices_with_pieces[diff_vertices_mask]
    diff_indices = np.where(diff_vertices_mask)[0]  
    
    pieces_mesh = diff_mesh.select_by_index(diff_indices)
    
    static_mesh  = static_mesh + pieces_mesh
    static_mesh  = static_mesh + deleted_mesh
    
    o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0{t:03}.obj', mesh_remove_face_number, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\static/mesh_0{t:03}_raw.obj', static_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    

#o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0000', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)


{(4, 0, 4), (2, 2, 2), (0, 1, 3), (2, 1, 3), (0, 3, 3), (4, 4, 4), (1, 2, 2), (1, 3, 3), (2, 3, 0), (2, 4, 5), (0, 2, 2), (1, 1, 3), (2, 1, 2), (0, 1, 2), (4, 4, 3), (0, 2, 3), (1, 1, 2), (5, 4, 0), (0, 5, 2), (0, 1, 1), (2, 2, 3), (0, 3, 1), (1, 2, 0), (1, 2, 3), (3, 3, 0), (4, 3, 0), (1, 1, 4)}
[Open3D WARNING] Write OBJ can not include triangle normals.
31
6
[Open3D WARNING] Write OBJ can not include triangle normals.
38
7
[Open3D WARNING] Write OBJ can not include triangle normals.
38
4
[Open3D WARNING] Write OBJ can not include triangle normals.
42
6
[Open3D WARNING] Write OBJ can not include triangle normals.
45
6
[Open3D WARNING] Write OBJ can not include triangle normals.
36
8
[Open3D WARNING] Write OBJ can not include triangle normals.
42
6
[Open3D WARNING] Write OBJ can not include triangle normals.
38
5
[Open3D WARNING] Write OBJ can not include triangle normals.
32
6
[Open3D WARNING] Write OBJ can not include triangle normals.
47
9


## Chatting

In [62]:
import open3d as o3d
import numpy as np
mesh1 = r"G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh/chatting_100.obj"
mesh7 = r"G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh/chatting_119.obj"
# Parameters
num_blocks = [6, 6, 6]  
dynamic_threshold = 0.95
comparison_tolerance = 0.000005
mesh_files = []
#for i in range (0, 10):
#    mesh_i = fr"G:\ChromeDownloads\Raw\Mesh/preprocessed/answering_questions_8{i:01}.obj"
#    mesh_files.append(mesh_i)
mesh_files = [mesh1, mesh7]  
meshes = [o3d.io.read_triangle_mesh(file) for file in mesh_files]

aabb = meshes[0].get_axis_aligned_bounding_box()
for mesh in meshes[1:]:
    aabb += mesh.get_axis_aligned_bounding_box()

min_bound = aabb.min_bound
max_bound = aabb.max_bound
block_size = [(max_bound[i] - min_bound[i]) / num_blocks[i] for i in range(3)]

def get_block_index(point):
    return tuple(int((point[i] - min_bound[i]) / block_size[i]) for i in range(3))


def is_block_dynamic(points1, points2, threshold, tolerance):
    if not len(points1) or not len(points2):  
        return True
    
    points1 = np.array(points1)
    points2 = np.array(points2)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points2)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    differing_points = 0

    for point in points1:
        [k, idx, dist] = pcd_tree.search_knn_vector_3d(point, 1)
        if k == 0 or dist[0] > tolerance: 
            differing_points += 1

    #print(differing_points)
    return (differing_points / len(points1)) > threshold

occupancy_maps = [{} for _ in meshes]

for t, mesh in enumerate(meshes):
    points = np.asarray(mesh.vertices)
    for point in points:
        block_index = get_block_index(point)
        if block_index not in occupancy_maps[t]:
            occupancy_maps[t][block_index] = []
        occupancy_maps[t][block_index].append(point)
 
all_blocks = set.union(*(set(occupancy.keys()) for occupancy in occupancy_maps))
dynamic_blocks = set()
static_blocks = set()

for block in all_blocks:
    is_dynamic = False
    for t in range(1, len(meshes)): # there is a bug...
        if is_block_dynamic(
            occupancy_maps[0].get(block, []),
            occupancy_maps[len(meshes)-1].get(block, []),
            threshold=dynamic_threshold,
            tolerance=comparison_tolerance,
        ):
            is_dynamic = True
            break
    if is_dynamic:
        dynamic_blocks.add(block)
    else:
        static_blocks.add(block)

print(dynamic_blocks)
for t, mesh in enumerate(meshes):
    static_points = []
    dynamic_points = []
    points = np.asarray(mesh.vertices)
    for i, point in enumerate(points):
        block_index = get_block_index(point)
        if block_index in static_blocks:
            static_points.append(i)
        elif block_index in dynamic_blocks:
            dynamic_points.append(i)
            
    dynamic_mesh, static_mesh = get_selected_parts(mesh, dynamic_points, static_points)
    
    o3d.visualization.draw_geometries([dynamic_mesh])
    o3d.io.write_triangle_mesh(fr'G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh\dynamic/mesh_0{t:03}_raw.obj', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

    
    planar_groups = find_planar_surfaces(dynamic_mesh, tolerance_deg=100)
    print(planar_groups.__len__())
    
    planar_meshes = [extract_mesh_from_group(dynamic_mesh, group) for group in planar_groups]
    
    deleted_mesh = o3d.geometry.TriangleMesh()
    for i, planar_mesh in enumerate(planar_meshes):
        deleted_mesh += planar_mesh
        
    #o3d.visualization.draw_geometries([deleted_mesh])

    o3d.io.write_triangle_mesh(fr'G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh\dynamic/mesh_0{t:03}_deleted.obj', deleted_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    mesh_raw = o3d.io.read_triangle_mesh(fr'G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh\dynamic\mesh_0{t:03}_raw.obj')
    mesh = o3d.io.read_triangle_mesh(fr'G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh\dynamic\mesh_0{t:03}_deleted.obj')
    vertices_raw = np.asarray(mesh_raw.vertices)
    vertices = np.asarray(mesh.vertices)
    
    diff_vertices_mask = np.any(vertices_raw[:, None] != vertices, axis=2).all(axis=1)
    
    diff_vertices = vertices_raw[diff_vertices_mask]
    diff_indices = np.where(diff_vertices_mask)[0]  
    
    diff_mesh = mesh_raw.select_by_index(diff_indices)
    #print(diff_mesh)
    
    o3d.io.write_triangle_mesh(fr'G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh\dynamic/mesh_0{t:03}.obj', diff_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    static_mesh  = static_mesh + deleted_mesh
    o3d.io.write_triangle_mesh(fr'G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh\static/mesh_0{t:03}_raw.obj', static_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    

#o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0000', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)


{(2, 0, 2), (2, 1, 0), (0, 3, 6), (3, 2, 4), (3, 1, 2), (1, 3, 0), (3, 4, 4), (3, 3, 2), (1, 0, 1), (2, 3, 3), (3, 0, 3), (0, 5, 3), (1, 5, 0), (0, 6, 4), (2, 2, 4), (0, 1, 2), (3, 1, 4), (1, 4, 4), (5, 5, 5), (3, 2, 3), (3, 4, 3), (3, 3, 4), (2, 3, 2), (4, 3, 4), (0, 5, 5), (6, 1, 0), (2, 0, 3), (2, 2, 3), (1, 2, 0), (3, 1, 3), (3, 2, 2), (1, 4, 0), (3, 3, 3), (5, 5, 1), (3, 4, 2), (2, 3, 4), (0, 2, 2), (0, 5, 4)}
[Open3D WARNING] Write OBJ can not include triangle normals.
22
8
[Open3D WARNING] Write OBJ can not include triangle normals.
31
7


## Drinking

In [10]:
import open3d as o3d
import numpy as np
#mesh1 = r"G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh/chatting_100.obj"
#mesh7 = r"G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh/chatting_119.obj"
# Parameters
num_blocks = [8, 8, 6]  
dynamic_threshold = 0.92
comparison_tolerance = 0.00003
mesh_files = []
for i in range (1, 8):
    mesh_i = fr"G:\VS2022Projects\arap-volume-tracking-main\data\scene\meshes\good/mesh_0{i:03}.obj"
    mesh_files.append(mesh_i)
#mesh_files = [mesh1, mesh7]  
meshes = [o3d.io.read_triangle_mesh(file) for file in mesh_files]

aabb = meshes[0].get_axis_aligned_bounding_box()
for mesh in meshes[1:]:
    aabb += mesh.get_axis_aligned_bounding_box()

min_bound = aabb.min_bound
max_bound = aabb.max_bound
block_size = [(max_bound[i] - min_bound[i]) / num_blocks[i] for i in range(3)]

def get_block_index(point):
    return tuple(int((point[i] - min_bound[i]) / block_size[i]) for i in range(3))


def is_block_dynamic(points1, points2, threshold, tolerance):
    if not len(points1) or not len(points2):  
        return True
    
    points1 = np.array(points1)
    points2 = np.array(points2)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points2)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    differing_points = 0

    for point in points1:
        [k, idx, dist] = pcd_tree.search_knn_vector_3d(point, 1)
        if k == 0 or dist[0] > tolerance: 
            differing_points += 1

    #print(differing_points)
    return (differing_points / len(points1)) > threshold

occupancy_maps = [{} for _ in meshes]

for t, mesh in enumerate(meshes):
    points = np.asarray(mesh.vertices)
    for point in points:
        block_index = get_block_index(point)
        if block_index not in occupancy_maps[t]:
            occupancy_maps[t][block_index] = []
        occupancy_maps[t][block_index].append(point)
 
all_blocks = set.union(*(set(occupancy.keys()) for occupancy in occupancy_maps))
dynamic_blocks = set()
static_blocks = set()

for block in all_blocks:
    is_dynamic = False
    for t in range(1, len(meshes)): # there is a bug...
        if is_block_dynamic(
            occupancy_maps[0].get(block, []),
            occupancy_maps[len(meshes)-1].get(block, []),
            threshold=dynamic_threshold,
            tolerance=comparison_tolerance,
        ):
            is_dynamic = True
            break
    if is_dynamic:
        dynamic_blocks.add(block)
    else:
        static_blocks.add(block)

print(dynamic_blocks)
for t, mesh in enumerate(meshes):
    static_points = []
    dynamic_points = []
    points = np.asarray(mesh.vertices)
    for i, point in enumerate(points):
        block_index = get_block_index(point)
        if block_index in static_blocks:
            static_points.append(i)
        elif block_index in dynamic_blocks:
            dynamic_points.append(i)
            
    dynamic_mesh, static_mesh = get_selected_parts(mesh, dynamic_points, static_points)
    
    #o3d.visualization.draw_geometries([dynamic_mesh])
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic/mesh_0{t:03}_raw.obj', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

    
    planar_groups = find_planar_surfaces(dynamic_mesh, tolerance_deg=170)
    print(planar_groups.__len__())
    
    planar_meshes = [extract_mesh_from_group(dynamic_mesh, group) for group in planar_groups]
    
    deleted_mesh = o3d.geometry.TriangleMesh()
    for i, planar_mesh in enumerate(planar_meshes):
        deleted_mesh += planar_mesh
        
    #o3d.visualization.draw_geometries([deleted_mesh])

    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic/mesh_0{t:03}_deleted.obj', deleted_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    mesh_raw = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic\mesh_0{t:03}_raw.obj')
    mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic\mesh_0{t:03}_deleted.obj')
    vertices_raw = np.asarray(mesh_raw.vertices)
    vertices = np.asarray(mesh.vertices)
    
    diff_vertices_mask = np.any(vertices_raw[:, None] != vertices, axis=2).all(axis=1)
    
    diff_vertices = vertices_raw[diff_vertices_mask]
    diff_indices = np.where(diff_vertices_mask)[0]  
    
    diff_mesh = mesh_raw.select_by_index(diff_indices)
    #print(diff_mesh)
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic/mesh_0{t:03}_before_delete_pieces.obj', diff_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic/mesh_0{t:03}_before_delete_pieces.obj')
    ms.meshing_remove_connected_component_by_face_number(mincomponentsize = 2500)
    ms.save_current_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic/mesh_0{t:03}.obj')

    mesh_remove_face_number = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic\mesh_0{t:03}.obj')
    #o3d.visualization.draw_geometries([mesh_remove_face_number])
    
    vertices_with_pieces = np.asarray(diff_mesh.vertices)
    vertices_without_pieces = np.asarray(mesh_remove_face_number.vertices)
    
    diff_vertices_mask = np.any(vertices_with_pieces[:, None] != vertices_without_pieces, axis=2).all(axis=1)
    
    diff_vertices = vertices_with_pieces[diff_vertices_mask]
    diff_indices = np.where(diff_vertices_mask)[0]  
    
    pieces_mesh = diff_mesh.select_by_index(diff_indices)
    
    static_mesh  = static_mesh + pieces_mesh
    static_mesh  = static_mesh + deleted_mesh
    
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic/mesh_0{t:03}.obj', mesh_remove_face_number, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/mesh_0{t:03}_raw.obj', static_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

#o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0000', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)


{(7, 4, 1), (5, 4, 2), (0, 6, 5), (0, 7, 4), (0, 1, 0), (1, 6, 3), (4, 6, 4), (1, 2, 5), (0, 0, 1), (2, 4, 2), (4, 3, 2), (8, 4, 2), (1, 0, 4), (1, 1, 3), (5, 4, 1), (1, 6, 2), (2, 2, 4), (2, 0, 4), (4, 6, 3), (1, 4, 1), (1, 3, 2), (0, 0, 0), (6, 4, 2), (7, 4, 2), (1, 0, 3), (0, 4, 6), (0, 7, 5), (1, 5, 2), (6, 1, 3), (1, 0, 2), (4, 4, 2), (0, 8, 0), (1, 4, 0), (3, 4, 2), (2, 4, 3), (6, 4, 1)}
[Open3D WARNING] Write OBJ can not include triangle normals.
24
19
[Open3D WARNING] Write OBJ can not include triangle normals.
25
22
[Open3D WARNING] Write OBJ can not include triangle normals.
26
24
[Open3D WARNING] Write OBJ can not include triangle normals.
24
20
[Open3D WARNING] Write OBJ can not include triangle normals.
24
22
[Open3D WARNING] Write OBJ can not include triangle normals.
24
22
[Open3D WARNING] Write OBJ can not include triangle normals.
28
25


In [103]:
mesh1 = o3d.io.read_triangle_mesh(fr"G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic/mesh_0001.obj")
mesh1d = o3d.io.read_triangle_mesh(fr"G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic/mesh_0001_deleted.obj")
mesh1.compute_vertex_normals()
mesh1d.compute_vertex_normals()
mesh1d.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([mesh1, mesh1d])

## sitting

In [13]:
import open3d as o3d
import numpy as np
#mesh1 = r"G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh/chatting_100.obj"
#mesh7 = r"G:\PycharmProjects\scene_compression\dataset\Chatting\Mesh/chatting_119.obj"
# Parameters
num_blocks = [8, 8, 6]  
dynamic_threshold = 0.95
comparison_tolerance = 0.00003
mesh_files = []
for i in range (6, 16):
    mesh_i = fr"G:\ChromeDownloads\4_cam_sync\mesh/0{i:03}.obj"
    mesh_files.append(mesh_i)
#mesh_files = [mesh1, mesh7]  
meshes = [o3d.io.read_triangle_mesh(file) for file in mesh_files]

aabb = meshes[0].get_axis_aligned_bounding_box()
for mesh in meshes[1:]:
    aabb += mesh.get_axis_aligned_bounding_box()

min_bound = aabb.min_bound
max_bound = aabb.max_bound
block_size = [(max_bound[i] - min_bound[i]) / num_blocks[i] for i in range(3)]

def get_block_index(point):
    return tuple(int((point[i] - min_bound[i]) / block_size[i]) for i in range(3))


def is_block_dynamic(points1, points2, threshold, tolerance):
    if not len(points1) or not len(points2):  
        return True
    
    points1 = np.array(points1)
    points2 = np.array(points2)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points2)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    differing_points = 0

    for point in points1:
        [k, idx, dist] = pcd_tree.search_knn_vector_3d(point, 1)
        if k == 0 or dist[0] > tolerance: 
            differing_points += 1

    #print(differing_points)
    return (differing_points / len(points1)) > threshold

occupancy_maps = [{} for _ in meshes]

for t, mesh in enumerate(meshes):
    points = np.asarray(mesh.vertices)
    for point in points:
        block_index = get_block_index(point)
        if block_index not in occupancy_maps[t]:
            occupancy_maps[t][block_index] = []
        occupancy_maps[t][block_index].append(point)
 
all_blocks = set.union(*(set(occupancy.keys()) for occupancy in occupancy_maps))
dynamic_blocks = set()
static_blocks = set()

for block in all_blocks:
    is_dynamic = False
    for t in range(1, len(meshes)): # there is a bug...
        if is_block_dynamic(
            occupancy_maps[0].get(block, []),
            occupancy_maps[len(meshes)-1].get(block, []),
            threshold=dynamic_threshold,
            tolerance=comparison_tolerance,
        ):
            is_dynamic = True
            break
    if is_dynamic:
        dynamic_blocks.add(block)
    else:
        static_blocks.add(block)

print(dynamic_blocks)
for t, mesh in enumerate(meshes):
    static_points = []
    dynamic_points = []
    points = np.asarray(mesh.vertices)
    for i, point in enumerate(points):
        block_index = get_block_index(point)
        if block_index in static_blocks:
            static_points.append(i)
        elif block_index in dynamic_blocks:
            dynamic_points.append(i)
            
    dynamic_mesh, static_mesh = get_selected_parts(mesh, dynamic_points, static_points)
    
    #o3d.visualization.draw_geometries([dynamic_mesh])
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\sitting\dynamic/mesh_0{t:03}_raw.obj', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

    
    planar_groups = find_planar_surfaces(dynamic_mesh, tolerance_deg=170)
    print(planar_groups.__len__())
    
    planar_meshes = [extract_mesh_from_group(dynamic_mesh, group) for group in planar_groups]
    
    deleted_mesh = o3d.geometry.TriangleMesh()
    for i, planar_mesh in enumerate(planar_meshes):
        deleted_mesh += planar_mesh
        
    #o3d.visualization.draw_geometries([deleted_mesh])

    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\sitting\dynamic/mesh_0{t:03}_deleted.obj', deleted_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    mesh_raw = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\sitting\dynamic\mesh_0{t:03}_raw.obj')
    mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\sitting\dynamic\mesh_0{t:03}_deleted.obj')
    vertices_raw = np.asarray(mesh_raw.vertices)
    vertices = np.asarray(mesh.vertices)
    
    diff_vertices_mask = np.any(vertices_raw[:, None] != vertices, axis=2).all(axis=1)
    
    diff_vertices = vertices_raw[diff_vertices_mask]
    diff_indices = np.where(diff_vertices_mask)[0]  
    
    diff_mesh = mesh_raw.select_by_index(diff_indices)
    #print(diff_mesh)
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\sitting\dynamic/mesh_0{t:03}_before_delete_pieces.obj', diff_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\sitting\dynamic/mesh_0{t:03}_before_delete_pieces.obj')
    ms.meshing_remove_connected_component_by_face_number(mincomponentsize = 1500)
    ms.save_current_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\sitting\dynamic/mesh_0{t:03}.obj')

    mesh_remove_face_number = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\sitting\dynamic\mesh_0{t:03}.obj')
    #o3d.visualization.draw_geometries([mesh_remove_face_number])
    
    vertices_with_pieces = np.asarray(diff_mesh.vertices)
    vertices_without_pieces = np.asarray(mesh_remove_face_number.vertices)
    
    diff_vertices_mask = np.any(vertices_with_pieces[:, None] != vertices_without_pieces, axis=2).all(axis=1)
    
    diff_vertices = vertices_with_pieces[diff_vertices_mask]
    diff_indices = np.where(diff_vertices_mask)[0]  
    
    pieces_mesh = diff_mesh.select_by_index(diff_indices)
    
    static_mesh  = static_mesh + pieces_mesh
    static_mesh  = static_mesh + deleted_mesh
    
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\sitting\dynamic/mesh_0{t:03}.obj', mesh_remove_face_number, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\sitting\static/mesh_0{t:03}_raw.obj', static_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

#o3d.io.write_triangle_mesh(fr'G:\ChromeDownloads\Raw\Mesh\dynamic/mesh_0000', dynamic_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)


{(6, 7, 3), (1, 5, 4), (2, 0, 5), (1, 6, 6), (4, 4, 1), (7, 0, 0), (4, 5, 0), (0, 3, 3), (4, 6, 1), (6, 6, 1), (2, 7, 4), (1, 2, 2), (1, 3, 3), (3, 3, 5), (7, 7, 2), (2, 3, 3), (7, 2, 0), (8, 7, 1), (7, 4, 0), (7, 6, 0), (1, 6, 5), (3, 7, 5), (2, 0, 4), (4, 4, 0), (0, 3, 2), (4, 6, 0), (1, 3, 2), (5, 7, 2), (7, 7, 1), (2, 3, 2), (7, 1, 0), (1, 1, 5), (7, 3, 0), (6, 7, 1), (5, 6, 0), (6, 0, 1), (7, 6, 2), (4, 8, 3), (1, 5, 5), (2, 2, 3), (3, 7, 4), (1, 7, 5), (7, 0, 1), (4, 5, 1), (2, 7, 5), (1, 2, 3), (7, 5, 0), (3, 3, 3), (2, 6, 3), (7, 7, 0), (0, 2, 2), (6, 4, 1), (1, 0, 5)}
[Open3D WARNING] Write OBJ can not include triangle normals.
26
19
[Open3D WARNING] Write OBJ can not include triangle normals.
23
18
[Open3D WARNING] Write OBJ can not include triangle normals.
19
13
[Open3D WARNING] Write OBJ can not include triangle normals.
24
22
[Open3D WARNING] Write OBJ can not include triangle normals.
12
10
[Open3D WARNING] Write OBJ can not include triangle normals.
19
15
[Open3D WARNIN